In [1]:
import numpy as np 
import pandas as pd
import math

# plots
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

In [2]:
filenames = ['properties/properati-AR-2016-03-01-properties-sell-six_months.csv',
            'properties/properati-AR-2016-04-01-properties-sell.csv',
            'properties/properati-AR-2016-05-01-properties-sell.csv',
            'properties/properati-AR-2016-06-01-properties-sell.csv',
            'properties/properati-AR-2016-07-01-properties-sell.csv',
            'properties/properati-AR-2016-08-01-properties-sell.csv',
            'properties/properati-AR-2016-09-01-properties-sell.csv',
            'properties/properati-AR-2016-10-01-properties-sell.csv',
            'properties/properati-AR-2016-11-01-properties-sell.csv',
            'properties/properati-AR-2016-12-01-properties-sell.csv', 
            'properties/properati-AR-2017-01-01-properties-sell.csv',
            'properties/properati-AR-2017-02-01-properties-sell.csv',
            'properties/properati-AR-2017-08-01-properties-sell-six_months.csv']

dataframes = []
for filename in filenames:
    dataframes.append(pd.read_csv(filename))

# SAQUE ESTO PARA HACER LA REGRESION A PARTIR DEL 2DO SEMESTRE DEL 2016
#dataframes.append(pd.read_csv('properties/properati-AR-2015-07-01-properties-sell.csv', skiprows=[81550]))
#dataframes.append(pd.read_csv('properties/properati-AR-2015-08-01-properties-sell.csv', skiprows=[75032]))
#dataframes.append(pd.read_csv('properties/properati-AR-2015-09-01-properties-sell.csv', skiprows=[54374]))
#dataframes.append(pd.read_csv('properties/properati-AR-2015-12-01-properties-sell.csv', skiprows=[101832]))
#Uno los dataframe
metadata = pd.DataFrame()
for dataframe in dataframes:
    metadata = pd.concat([metadata, dataframe]).drop_duplicates().reset_index(drop=True)


In [3]:
data = metadata.loc[(metadata.state_name.str.contains('Buenos Aires')) | (metadata.state_name.str.contains\
                ('Capital Federal'))  | (metadata.state_name.str.contains\
                ('Bs.As') ),:]
data.shape
# 1.283.741 entries antes, ahora 242290

(242290, 27)

In [4]:
test = pd.read_csv('properties/properati_dataset_testing_noprice.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 17 columns):
id                         14166 non-null int64
created_on                 14166 non-null object
property_type              14166 non-null object
operation                  14166 non-null object
place_name                 14166 non-null object
place_with_parent_names    14166 non-null object
country_name               14166 non-null object
state_name                 14166 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        11853 non-null float64
surface_covered_in_m2      13005 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14166 non-null object
dtypes: float64(6), int64(1), object(10)
memory usage: 1

In [5]:
test_visualiz = test.loc[:,['state_name', 'id']].groupby('state_name').agg(np.size)
test_visualiz.head(10)

,id
state_name,
Bs.As. G.B.A. Zona Norte,2
Bs.As. G.B.A. Zona Oeste,3101
Bs.As. G.B.A. Zona Sur,2215
Buenos Aires Interior,1
Capital Federal,8847


In [6]:
data['state_name'].tail(10)

296467        Bs.As. G.B.A. Zona Norte
296468        Bs.As. G.B.A. Zona Norte
296469        Bs.As. G.B.A. Zona Norte
296470        Bs.As. G.B.A. Zona Norte
296471        Bs.As. G.B.A. Zona Norte
296472                 Capital Federal
296473        Bs.As. G.B.A. Zona Norte
296474                 Capital Federal
296475    Buenos Aires Costa Atlántica
296476                 Capital Federal
Name: state_name, dtype: object

In [8]:
# Depuración del set de test
#Completo los valores NaN con promedios para realizar la predicción
import calendar 
prom_surface_total_in_m2 = test['surface_total_in_m2'].mean()
prom_surface_covered_in_m2 = test['surface_covered_in_m2'].mean()
prom_lat = test['lat'].mean()
prom_lon = test['lon'].mean()

test_float = test
test_float['surface_total_in_m2'].fillna(prom_surface_total_in_m2, inplace=True)
test_float['surface_covered_in_m2'].fillna(prom_surface_covered_in_m2, inplace=True)
test_float['lat'].fillna(prom_lat, inplace=True)
test_float['lon'].fillna(prom_lon, inplace=True)

test_float['fecha'] = pd.to_datetime(test_float['created_on'])
test_float['anio'] = test_float['fecha'].map(lambda x:x.year)
test_float['mes'] = test_float['fecha'].map(lambda x:x.month)

test_float.head(100)

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,fecha,anio,mes
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.000000,359.471588,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...,2017-08-24,2017,8
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,-34.629923,-58.465820,0.000000,359.471588,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...,2017-08-25,2017,8
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.000000,48.000000,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...,2017-08-01,2017,8
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2017-08-01,2017,8
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,51.000000,51.000000,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...,2017-08-01,2017,8
5,2263407,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2017-08-01,2017,8
6,2263408,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2017-08-01,2017,8
7,2263409,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2017-08-01,2017,8
8,2263410,2017-08-01,departamento,venta,Colegiales,|Argentina|Capital Federal|Colegiales|,Argentina,Capital Federal,"-34.5787213694,-58.4570952614",-34.578721,-58.457095,67.000000,67.000000,NaN,NaN,2600,EXCELENTE 2 AMB- APTO PROFESIO...,2017-08-01,2017,8
9,2263411,2017-08-01,departamento,venta,Almagro,|Argentina|Capital Federal|Almagro|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,47.000000,47.000000,NaN,NaN,NaN,IMPECABLE MONOAMBIENTE- DIVISI...,2017-08-01,2017,8


In [41]:
#FILTRO LOS DATOS ANOMALOS

data_float = data.loc[:, ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',\
                           'lat', 'lon']].dropna()

#filtro
data_float = \
data_float.loc[((data_float.surface_total_in_m2 >= 0.0) & (data_float.surface_total_in_m2 <= 1000000.0)) & \
               ((data_float.surface_covered_in_m2 >= 0.0) & (data_float.surface_covered_in_m2 <= 50000000.0)), :]


data_float.shape
#(68586, 6)

(104604, 5)

In [42]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, init='k-means++', )
points = kmeans.fit_transform(data_float)
#kmeans.cluster_centers_
points

array([[  2.79026919e+05,   4.05912216e+05,   5.73973885e+05,
          5.81934188e+06,   1.87165587e+06],
       [  3.20973313e+05,   1.00591223e+06,   2.60268188e+04,
          6.41934201e+06,   2.47165594e+06],
       [  2.90973473e+05,   9.75912326e+05,   3.97472614e+03,
          6.38934218e+06,   2.44165602e+06],
       ..., 
       [  2.55973438e+05,   9.40912304e+05,   3.89736360e+04,
          6.35434215e+06,   2.40665601e+06],
       [  2.55973500e+05,   9.40912332e+05,   3.89737311e+04,
          6.35434219e+06,   2.40665603e+06],
       [  2.89473474e+05,   9.74412326e+05,   5.47442602e+03,
          6.38784218e+06,   2.44015602e+06]])

In [43]:
len(kmeans.labels_)

104604

In [13]:
#y_pred = KMeans(n_clusters=3, random_state=random_state).fit_predict(X_varied)

In [44]:
kmeans.cluster_centers_

array([[  4.20973294e+05,   3.24612408e+02,   2.03190377e+02,
         -3.47066255e+01,  -5.85101998e+01],
       [  1.10591219e+06,   4.73844690e+02,   3.42495724e+02,
         -3.45612937e+01,  -5.84498230e+01],
       [  1.26026388e+05,   1.31743168e+02,   7.84396894e+01,
         -3.51094409e+01,  -5.84093428e+01],
       [  6.51934043e+06,   4.19508511e+03,   2.32551064e+03,
         -3.45608812e+01,  -5.84414327e+01],
       [  2.57165584e+06,   8.20066929e+02,   5.91931102e+02,
         -3.46257617e+01,  -5.84499580e+01]])

In [47]:
kmeans = KMeans(n_clusters=3, init='k-means++', )
kmeans.fit(data_float)
print kmeans.score(data_float)


-2.92216444445e+15


In [51]:
kmeans = KMeans(n_clusters=4, init='k-means++', )
kmeans.fit(data_float)
print kmeans.score(data_float)


-1.83760416745e+15


In [52]:
kmeans = KMeans(n_clusters=5, init='k-means++', )
kmeans.fit(data_float)
print kmeans.score(data_float)

-1.26697646675e+15


In [53]:
kmeans = KMeans(n_clusters=6, init='k-means++', )
points = kmeans.fit_transform(data_float)
print kmeans.score(data_float)


-8.45624251813e+14


In [54]:
kmeans = KMeans(n_clusters=7, init='k-means++', )
points = kmeans.fit_transform(data_float)
print kmeans.score(data_float)


-6.67627524676e+14


In [55]:
kmeans = KMeans(n_clusters=8, init='k-means++', )
points = kmeans.fit_transform(data_float)
print kmeans.score(data_float)


-5.23437339839e+14


In [56]:
kmeans = KMeans(n_clusters=9, init='k-means++', )
points = kmeans.fit_transform(data_float)
print kmeans.score(data_float)


-4.18081585803e+14


In [57]:
kmeans = KMeans(n_clusters=3, init='k-means++', )
points = kmeans.fit_transform(data_float)
points

array([[  5.35200724e+05,   2.31350136e+06,   1.43101971e+05],
       [  6.47996472e+04,   2.91350146e+06,   7.43101853e+05],
       [  3.47997686e+04,   2.88350156e+06,   7.13101961e+05],
       ..., 
       [  2.14308707e+02,   2.84850154e+06,   6.78101936e+05],
       [  2.38591945e+02,   2.84850156e+06,   6.78101970e+05],
       [  3.32997806e+04,   2.88200156e+06,   7.11601962e+05]])

In [58]:
len(points)

104604

In [ ]:
d = {'col1': points[:,0], 'col2': points[:,1], 'col3': points[:,2]}
points_data = pd.DataFrame(data=d)

data_float.reset_index(inplace=True)
data_float['KM1'] = points_data['col1']
data_float['KM2'] = points_data['col2']
data_float['KM3'] = points_data['col3']
data_float.tail()


fit_predict(X, y=None) me devuelve un array de 68586 con un número que corresponde al cluster al que pertenece cada punto. De la misma forma, puedo acceder a cluster_centers_ y eso es lo que le sumo a cada observacion.
OJO porque si K = 5 entonces le estoy agregando 5 dimensiones a todo esto!!!!

## Verifico cuál es el Koptimo usando Hierarchical 

## Trato de usar PCA

In [17]:
data_float_red = data_float
#data_float_red['PCA1'] = np.zeros(len(data_float_red['rooms']))
#data_float_red['PCA2'] = np.zeros(len(data_float_red['rooms']))
data_float_red

,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,lat,lon
0,700000.00,500.0,500.0,6.0,-34.643701,-58.651548
13,98000.00,135.0,135.0,4.0,-34.546433,-58.545713
14,430000.00,330.0,310.0,6.0,-34.345731,-58.855461
15,270000.00,0.0,218.0,4.0,-34.735512,-58.398874
16,72000.00,30.0,30.0,2.0,-34.649843,-58.657190
17,102000.00,47.0,47.0,2.0,-34.651906,-58.644602
19,189000.00,294.0,100.0,4.0,-34.767167,-58.215465
20,145000.00,79.0,70.0,3.0,-34.578408,-58.495869
21,220000.00,75.0,65.0,2.0,-34.401696,-58.640455
22,105000.00,0.0,41.0,1.0,-34.588197,-58.501429


In [25]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(data_float)
red = pca.transform(data_float)
#red
print(pca.singular_values_)

#[  1.02754952e+08   1.91422523e+07   3.11793772e+05]
# el tercer valor singular es considerablemente más chico que los dos anteriores. Por lo tanto trabajo con 
# dos dimensiones

[  1.02754952e+08   1.91422523e+07   3.11793772e+05]


In [26]:
pca = PCA(n_components=2)
pca.fit(data_float)
red = pca.transform(data_float)
print(pca.singular_values_)

[  1.02754952e+08   1.91422523e+07]


In [28]:
red

array([[ 691654.37503471,  133994.95255174],
       [-159638.84515886,  123595.01079413],
       [ 309845.21401464,  129322.35409132],
       ..., 
       [-230982.07099549, -172745.48472993],
       [ 198906.91416304, -167501.27349429],
       [-188558.65316519, -172229.77296627]])

In [27]:
d = {'col1': red[:,0], 'col2': red[:,1]}
red_data = pd.DataFrame(data=d)

data_float.reset_index(inplace=True)
data_float['PCA1'] = red_data['col1']
data_float['PCA2'] = red_data['col2']
data_float.tail()


,level_0,index,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,lat,lon,PCA1,PCA2
68581,68581,295459,50000.0,70.0,200.0,3.0,-38.030114,-57.574442,-223911.493716,-172657.189361
68582,68582,295460,74000.0,39.0,39.0,2.0,-38.002849,-57.544908,-189972.937019,-172244.118969
68583,68583,295461,45000.0,27.0,27.0,1.0,-37.996638,-57.550819,-230982.070995,-172745.484730
68584,68584,295462,349000.0,149.0,129.0,5.0,-37.099113,-56.840540,198906.914163,-167501.273494
68585,68585,295463,75000.0,390.0,135.0,4.0,-37.837561,-58.259803,-188558.653165,-172229.772966


## Intento depurar 'description'

In [4]:
#FILTRO LOS DATOS ANOMALOS

data_float = data.loc[:, ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',\
                          'rooms', 'lat', 'lon', 'description']].dropna()

#filtro
data_float = \
data_float.loc[((data_float.surface_total_in_m2 >= 0.0) & (data_float.surface_total_in_m2 <= 1000000.0)) & \
               ((data_float.surface_covered_in_m2 >= 0.0) & (data_float.surface_covered_in_m2 <= 50000000.0)) & \
               ((data_float.rooms >= 0.0) & (data_float.rooms <= 200.0)), :]


data_float.shape
#(68584, 7)

(68584, 7)

In [20]:
def eraseSpecialCharacters(x):
    result = ''.join(char for char in x if char.isalnum())
    return (result.lower())
                
data_float['descripcion'] = data_float['description']
data_float['descripcion'].apply(eraseSpecialCharacters)
data_float

,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,lat,lon,description,descripcion
0,700000.00,500.0,500.0,6.0,-34.643701,-58.651548,"Venta - Chalet - LOTE 730 M2 - 17,32x43,30mts....","Venta - Chalet - LOTE 730 M2 - 17,32x43,30mts...."
13,98000.00,135.0,135.0,4.0,-34.546433,-58.545713,"P/H al frente de 4 ambientes, living , cocina ...","P/H al frente de 4 ambientes, living , cocina ..."
14,430000.00,330.0,310.0,6.0,-34.345731,-58.855461,Casa de alto nivel de construcción sobre lote ...,Casa de alto nivel de construcción sobre lote ...
15,270000.00,0.0,218.0,4.0,-34.735512,-58.398874,"Jardín al frente con rejas, entrada de auto y ...","Jardín al frente con rejas, entrada de auto y ..."
16,72000.00,30.0,30.0,2.0,-34.649843,-58.657190,Chalet en ph al frente en excelente ubicacion!...,Chalet en ph al frente en excelente ubicacion!...
17,102000.00,47.0,47.0,2.0,-34.651906,-58.644602,Excelente departamento de 2 ambientes con coch...,Excelente departamento de 2 ambientes con coch...
19,189000.00,294.0,100.0,4.0,-34.767167,-58.215465,CASA A LA VENTA EN BERAZATEGUI.Casa sobre terr...,CASA A LA VENTA EN BERAZATEGUI.Casa sobre terr...
20,145000.00,79.0,70.0,3.0,-34.578408,-58.495869,Departamento en Venta en Villa Urquiza de 2 do...,Departamento en Venta en Villa Urquiza de 2 do...
21,220000.00,75.0,65.0,2.0,-34.401696,-58.640455,Intercontinental: a estrenar: dto de 2 ambient...,Intercontinental: a estrenar: dto de 2 ambient...
22,105000.00,0.0,41.0,1.0,-34.588197,-58.501429,VENTA DE MONOAMBIENTE DIVISIBLE A ESTRENAR EN ...,VENTA DE MONOAMBIENTE DIVISIBLE A ESTRENAR EN ...


In [117]:
def eraseSpecialCharacters(x):
    
    import re
    newlabel = re.sub('[^A-Za-z0-9@]+[\:.,:°\-\*]', '', x)
    newlabel = re.sub('[-:]', '', newlabel)
    newlabel = re.sub('[.,]', ' ', newlabel)
    newlabel = newlabel.lower()
    return newlabel
    
string = data_float['description'][0]
resultado = eraseSpecialCharacters(string)
print resultado

venta chalet lote 730 m2 17 32x43 30mts castelar chalet 3 plantas 500mts  cubiertos pb jardín al frente  hall  living comedor  cocina comedor diario  toilette  2 baños completos  lavadero  escritorio  dep  de servicio  garage para 2 autos  quincho  piscina  todos los servicios  fondo libre parquizado 1p 3 dormitorios con placard 1 en suite con vestidor  baño completo 2p playroom oyhanarte propiedades oyhanarte propiedades


In [102]:
print string

Venta - Chalet - LOTE 730 M2 - 17,32x43,30mts. - Castelar.Chalet 3 plantas- 500mts. cubiertos.PB: Jardín al frente, hall, living comedor, cocina comedor diario, toilette, 2 baños completos, lavadero, escritorio, dep. de servicio, garage para 2 autos, quincho, piscina, todos los servicios, fondo libre parquizado.1°P: 3 Dormitorios con placard 1 en suite con vestidor, baño completo.2°P: Playroom.Oyhanarte Propiedades. -  - OYHANARTE PROPIEDADES


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
